<a href="https://colab.research.google.com/github/massirism/Data-Analytics-CUC/blob/main/RINITIS_DESPLIEGUE_Massiris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#Cargamos el modelo
import pickle
filename = 'modelo-clas-tree.pkl'
modelTree,labelencoder,variables = pickle.load(open(filename, 'rb')) #read

In [ ]:
#Cargar datos
df=pd.read_excel("Medicamentos rinitis.xlsx",sheet_name="predicciones futuras")
df.head()

,ID,EDAD,SEXO,PRESIÓN SANGUÍNEA,COLESTEROL,SODIO,POTASIO
0,1,64,M,HIGH,NORMAL,0.739914,0.035349
1,2,58,M,HIGH,HIGH,0.769090,0.040497
2,3,145,M,HIGH,HIGH,0.534060,0.066666
3,4,72,M,LOW,HIGH,0.547347,0.033560
4,5,72,M,LOW,HIGH,0.504820,0.074573


Hay una edad de 145 años, quitaría toda la fila pero teniendo tan pocos datos prefiero tomar con ligereza ese problema. En otras palabras, lo ignoraré.

In [ ]:
ids=df["ID"].value_counts() #Cuenta la frecuencia en la que se repite cada ID
ids[ids>1]

Series([], Name: ID, dtype: int64)

In [ ]:
df=df.drop("ID",axis=1) #Eliminamos el ID por ser irrelevante, axis=1 indica que es una columna.df_train=df_train.drop("ID",axis=1) 

In [ ]:
#Se realiza la preparación
data_preparada=df.copy()
data_preparada = pd.get_dummies(data_preparada, columns=['PRESIÓN SANGUÍNEA',"COLESTEROL","SEXO"], drop_first=False)
data_preparada.head()

,EDAD,SODIO,POTASIO,PRESIÓN SANGUÍNEA_HIGH,PRESIÓN SANGUÍNEA_LOW,PRESIÓN SANGUÍNEA_NORMAL,COLESTEROL_HIGH,COLESTEROL_NORMAL,SEXO_F,SEXO_M
0,64,0.739914,0.035349,1,0,0,0,1,0,1
1,58,0.769090,0.040497,1,0,0,1,0,0,1
2,145,0.534060,0.066666,1,0,0,1,0,0,1
3,72,0.547347,0.033560,0,1,0,1,0,0,1
4,72,0.504820,0.074573,0,1,0,1,0,0,1


In [ ]:
#Se adicionan las columnas faltantes
data_preparada=data_preparada.reindex(columns=variables,fill_value=0)
data_preparada.head()

,EDAD,SODIO,POTASIO,PRESIÓN SANGUÍNEA_HIGH,PRESIÓN SANGUÍNEA_LOW,PRESIÓN SANGUÍNEA_NORMAL,SEXO_M,COLESTEROL_NORMAL
0,64,0.739914,0.035349,1,0,0,1,1
1,58,0.769090,0.040497,1,0,0,1,0
2,145,0.534060,0.066666,1,0,0,1,0
3,72,0.547347,0.033560,0,1,0,1,0
4,72,0.504820,0.074573,0,1,0,1,0


#Predicción decisión tree

In [ ]:
#Hacemos la predicción
Y_Tree = modelTree.predict(data_preparada)
print(Y_Tree)
df['Prediccion_Tree']=labelencoder.inverse_transform(Y_Tree)
df


[1 4 1 1 2 4 2 2 3 3 3]


,EDAD,SEXO,PRESIÓN SANGUÍNEA,COLESTEROL,SODIO,POTASIO,Prediccion_Tree
0,64,M,HIGH,NORMAL,0.739914,0.035349,drugB
1,58,M,HIGH,HIGH,0.769090,0.040497,drugY
2,145,M,HIGH,HIGH,0.534060,0.066666,drugB
3,72,M,LOW,HIGH,0.547347,0.033560,drugB
4,72,M,LOW,HIGH,0.504820,0.074573,drugC
5,46,F,HIGH,HIGH,0.773569,0.022302,drugY
6,56,F,LOW,HIGH,0.848774,0.073380,drugC
7,16,M,LOW,HIGH,0.743021,0.061886,drugC
8,52,M,NORMAL,HIGH,0.549945,0.055581,drugX
9,23,M,NORMAL,NORMAL,0.784520,0.055959,drugX


#Despliegue del modelo regression tree with knn

In [ ]:
#Cargamos el modelo
import pickle
filename = 'modelo-clas-tree-knn.pkl'
modelTree,model_knn,labelencoder,variables,min_max_scaler = pickle.load(open(filename, 'rb')) #read

In [ ]:
#Usamos la data que ya está preparada y realizo la normalización de la edad en esta misma
#Se normaliza la edad

data_preparada[["EDAD"]]=min_max_scaler.transform(data_preparada[["EDAD"]])
data_preparada

,EDAD,SODIO,POTASIO,PRESIÓN SANGUÍNEA_HIGH,PRESIÓN SANGUÍNEA_LOW,PRESIÓN SANGUÍNEA_NORMAL,SEXO_M,COLESTEROL_NORMAL
0,0.833333,0.739914,0.035349,1,0,0,1,1
1,0.733333,0.769090,0.040497,1,0,0,1,0
2,2.183333,0.534060,0.066666,1,0,0,1,0
3,0.966667,0.547347,0.033560,0,1,0,1,0
4,0.966667,0.504820,0.074573,0,1,0,1,0
5,0.533333,0.773569,0.022302,1,0,0,0,0
6,0.700000,0.848774,0.073380,0,1,0,0,0
7,0.033333,0.743021,0.061886,0,1,0,1,0
8,0.633333,0.549945,0.055581,0,0,1,1,0
9,0.150000,0.784520,0.055959,0,0,1,1,1


In [ ]:
#Hacemos la prediccion con knn
Y_fut2=model_knn.predict(data_preparada)
df['Prediccion KNN']=labelencoder.inverse_transform(Y_fut2)
df

,EDAD,SEXO,PRESIÓN SANGUÍNEA,COLESTEROL,SODIO,POTASIO,Prediccion_Tree,Prediccion KNN
0,64,M,HIGH,NORMAL,0.739914,0.035349,drugB,drugY
1,58,M,HIGH,HIGH,0.769090,0.040497,drugY,drugB
2,145,M,HIGH,HIGH,0.534060,0.066666,drugB,drugB
3,72,M,LOW,HIGH,0.547347,0.033560,drugB,drugC
4,72,M,LOW,HIGH,0.504820,0.074573,drugC,drugC
5,46,F,HIGH,HIGH,0.773569,0.022302,drugY,drugA
6,56,F,LOW,HIGH,0.848774,0.073380,drugC,drugC
7,16,M,LOW,HIGH,0.743021,0.061886,drugC,drugC
8,52,M,NORMAL,HIGH,0.549945,0.055581,drugX,drugY
9,23,M,NORMAL,NORMAL,0.784520,0.055959,drugX,drugX


Tengo 5 resultados iguales y 5 distintos.
*  model_TREE------->ACCURACY=0.82
*  model_TREE+KNN--->ACCURACY=0.65
